<a href="https://colab.research.google.com/github/rajkumarshahu/AI-Colab/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

#Data Exploration and Cleaning

In [ ]:


# Load the data
data = pd.read_csv("/content/heart_2022_with_nans.csv", delimiter="\t")


# Display the first few rows of the dataset
print(data.head(10))

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing Values:")
print(missing_values)

# Data types of each column
print("\nData Types:")
print(data.dtypes)

# Basic statistics of the dataset
print("\nBasic Statistics:")
print(data.describe())

# Handling missing values
# For numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].mean())

# For categorical columns (assuming object type represents categorical columns)
categorical_cols = data.select_dtypes(include=['object']).columns
data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

# Verify if missing values have been handled
print("\nMissing Values After Handling:")
print(data.isnull().sum())



  State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0  Alabama,Female,Very good,0.0,0.0,Within past y...                                                                                                                                                                                                                                                                                                                                                                      

# Visualizing Data Distribution


In [ ]:
sex_column = data
sex_column

,"State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos"
0,"Alabama,Female,Very good,0.0,0.0,Within past y..."
1,"Alabama,Female,Excellent,0.0,0.0,,No,6.0,,No,N..."
2,"Alabama,Female,Very good,2.0,3.0,Within past y..."
3,"Alabama,Female,Excellent,0.0,0.0,Within past y..."
4,"Alabama,Female,Fair,2.0,0.0,Within past year (..."
...,...
445127,"Virgin Islands,Female,Good,0.0,3.0,Within past..."
445128,"Virgin Islands,Female,Excellent,2.0,2.0,Within..."
445129,"Virgin Islands,Female,Poor,30.0,30.0,5 or more..."
445130,"Virgin Islands,Male,Very good,0.0,0.0,Within p..."


In [ ]:
sns.histplot(data['Sex'], kde=True) # Replace 'column_name' with actual column names
plt.show()

KeyError: ignored